<a href="https://colab.research.google.com/github/fjve-v/analisis_everpeak/blob/main/Construyendo_un_pipeline_completo_para_limpieza_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
df = pd.read_csv("everpeak_retail.csv")

In [4]:
df["customer_age"] = df["customer_age"].replace(-999, pd.NA)
df["product_category"] = df["product_category"].replace("?", "unknown")

In [5]:
numeric_sentinels = [-999, 999] # sentinels que se reemplazan por pd.NA
df["customer_age"] = df["customer_age"].replace(numeric_sentinels , pd.NA)

text_sentinels = ["?"] # sentinels de texto
df["product_category"] = df["product_category"].replace(text_sentinels , "unknown")



In [6]:
def reemplazar_sentinels_global(df):
    numeric_sentinels = [-999, 999] # sentinels que se reemplazan por pd.NA
    df["customer_age"] = pd.to_numeric(df["customer_age"], errors="coerce")
    df["customer_age"] = df["customer_age"].replace(numeric_sentinels , pd.NA)

    text_sentinels = ["?"] # sentinels de texto
    df["product_category"] = df["product_category"].replace(text_sentinels,"unknown")
    return df

In [7]:
# función que pueda repetir el mismo proceso para distintas columnas numéricas o de texto agregando un bucle for.

def reemplazar_sentinels_global(df, numeric_cols, text_cols):
    numeric_sentinels = [-999, 999]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df[col] = df[col].replace(numeric_sentinels, pd.NA)

    text_sentinels = ["?"]
    for col in text_cols:
        df[col] = df[col].replace(text_sentinels, "unknown")
    return df

In [8]:
# establecer columnas a procesar
columnas_numericas = ["customer_age"]      # columnas numéricas a procesar
columnas_texto = ["product_category"]       # columnas texto a procesar


reemplazar_sentinels_global(df, columnas_numericas, columnas_texto)


df.info()
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          5008 non-null   int64  
 1   order_date        5000 non-null   object 
 2   customer_id       5008 non-null   int64  
 3   product_category  5008 non-null   object 
 4   price             5008 non-null   int64  
 5   quantity          5008 non-null   int64  
 6   order_value       5008 non-null   int64  
 7   payment_method    5008 non-null   object 
 8   city              4908 non-null   object 
 9   state             4908 non-null   object 
 10  customer_age      4833 non-null   float64
dtypes: float64(1), int64(5), object(5)
memory usage: 430.5+ KB
   order_id  order_date  customer_id product_category  price  quantity  \
0         1  2024-02-02         2616           Sports    269        50   
1         2  2024-10-10         1736          Grocery     66         0

####  Crear flags antes de imputar

In [11]:
df["age_missing_flag"] = df["customer_age"].isna().astype(int) # missing (que falta)
df["city_missing_flag"] = df["city"].isna().astype(int)
df["state_missing_flag"] = df["state"].isna().astype(int)

In [12]:
# función para crear columnas flags
def crear_flags(df, flags_cols):
    for col in flags_cols:
        nombre_flag = col + "_missing_flag"
        df[nombre_flag] = df[col].isna().astype(int)
    return df

# columnas a procesar
columnas_flags= ["customer_age", "city", "state"]


df = crear_flags(df, columnas_flags)
df.info()
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   order_id                   5008 non-null   int64  
 1   order_date                 5000 non-null   object 
 2   customer_id                5008 non-null   int64  
 3   product_category           5008 non-null   object 
 4   price                      5008 non-null   int64  
 5   quantity                   5008 non-null   int64  
 6   order_value                5008 non-null   int64  
 7   payment_method             5008 non-null   object 
 8   city                       4908 non-null   object 
 9   state                      4908 non-null   object 
 10  customer_age               4833 non-null   float64
 11  customer_age_missing_flag  5008 non-null   int64  
 12  city_missing_flag          5008 non-null   int64  
 13  state_missing_flag         5008 non-null   int64

#### Imputar valores ausentes según diagnóstico

In [10]:
# rellenamos los valores ausentes con la mediana
age_median = df["customer_age"].median() # determinacion de la mediana
print(age_median)

49.0


In [13]:
df["customer_age"] = df["customer_age"].fillna(age_median) # se rellena los datos faltantes con la mediana

In [14]:
median_fill_cols =["customer_age"]

for col in median_fill_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce") # convertimos a numerico con pd.to_numeric
    med = df[col].median() # evaluamos la mediana
    df[col] = df[col].fillna(med) # imputamos com la mediana

In [15]:
df["city"] = df["city"].fillna("unknown")
df["state"] = df["state"].fillna("unknown")

In [16]:
# Nuevamente, agregamos un bucle que recorre las columnas a rellenar, para automatizar este código:

In [17]:
fill_unknown_cols = ["city", "state"]   # lista de columnas a rellenar con unknown
for col in fill_unknown_cols:
    df[col] = df[col].fillna("unknown")

In [18]:
df = df.dropna(subset=["order_date"])

In [19]:
date_drop_cols= ["order_date"]

for col in date_drop_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")
    df = df.dropna(subset=[col]).reset_index(drop=True)

#### Crear función  imputar según diagnóstico

In [21]:
# crear función para imputar ausentes
def imputar_segun_diagnostico(df, median_fill_cols, fill_unknown_cols, date_drop_cols):
    # rellenar con la mediana en columnas numéricas
    for col in median_fill_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        med = df[col].median()
        df[col] = df[col].fillna(med)

    # rellenar con texto "unknown" en columnas categóricas
    for col in fill_unknown_cols:
        df[col] = df[col].fillna("unknown")

    # eliminar registros con valores ausentes en columnas tipo fecha
    for col in date_drop_cols:
        df[col] = pd.to_datetime(df[col], errors="coerce")
        df = df.dropna(subset=[col]).reset_index(drop=True)
    return df

In [22]:
# establecer columnas a procesar
cols_imputar_mediana = ["customer_age"]
cols_imputar_unknown = ["city", "state"]
cols_imputar_fecha = ["order_date"]

# aplicar función y mostrar resultados
df = imputar_segun_diagnostico(df, median_fill_cols, fill_unknown_cols, date_drop_cols)
df.info()
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   order_id                   5000 non-null   int64         
 1   order_date                 5000 non-null   datetime64[ns]
 2   customer_id                5000 non-null   int64         
 3   product_category           5000 non-null   object        
 4   price                      5000 non-null   int64         
 5   quantity                   5000 non-null   int64         
 6   order_value                5000 non-null   int64         
 7   payment_method             5000 non-null   object        
 8   city                       5000 non-null   object        
 9   state                      5000 non-null   object        
 10  customer_age               5000 non-null   float64       
 11  customer_age_missing_flag  5000 non-null   int64         
 12  city_m

#### Combinando funciones en un solo pipeline

In [23]:
def clean_data(df, numeric_cols, text_cols):
    # Sentinels: reemplazar marcadores inválidos por NaN
    df = reemplazar_sentinels_global(df, numeric_cols, text_cols)

    return df

In [24]:
#  agregaamos la función crear_flags con su entrada,

def clean_data(df, numeric_cols, text_cols, flags_cols):
    # Sentinels: reemplazar marcadores inválidos por NaN
    df = reemplazar_sentinels_global(df, numeric_cols, text_cols)

    # Flags: crear banderas antes de imputar
    df = crear_flags(df, flags_cols)

    return df

In [25]:
#  función imputar_segun_diagnostico con todas las listas de columnas que necesita:


def clean_data(df,numeric_cols, text_cols, flags_cols,
    median_fill_cols, fill_unknown_cols, date_drop_cols):

    # Sentinels: reemplazar marcadores inválidos por NaN
    df = reemplazar_sentinels_global(df, numeric_cols, text_cols)

    # Flags: crear banderas antes de imputar
    df = crear_flags(df, flags_cols)

    # Imputaciones / drops finales
    df = imputar_segun_diagnostico(df,median_fill_cols,fill_unknown_cols,date_drop_cols)
    return df

In [27]:
# main_pipeline.py

df = pd.read_csv("everpeak_retail.csv")

# reemplazar_sentinels
columnas_numericas = ["customer_age"]
columnas_texto = ["product_category"]

# crear_flags
columnas_flags = ["customer_age", "city", "state"]

# imputar_segun_diagnostico
cols_imputar_mediana = ["customer_age"]
cols_imputar_unknown = ["city", "state"]
cols_imputar_fecha = ["order_date"]

# Crear df_clean
df_clean = clean_data(df, columnas_numericas, columnas_texto, columnas_flags,
    cols_imputar_mediana, cols_imputar_unknown, cols_imputar_fecha)

# guardar df_clean en un CSV nuevo
df_clean.to_csv("everpeak_clean.csv", index=False)

In [28]:
df = pd.read_csv("everpeak_clean.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   order_id                   5000 non-null   int64  
 1   order_date                 5000 non-null   object 
 2   customer_id                5000 non-null   int64  
 3   product_category           5000 non-null   object 
 4   price                      5000 non-null   int64  
 5   quantity                   5000 non-null   int64  
 6   order_value                5000 non-null   int64  
 7   payment_method             5000 non-null   object 
 8   city                       5000 non-null   object 
 9   state                      5000 non-null   object 
 10  customer_age               5000 non-null   float64
 11  customer_age_missing_flag  5000 non-null   int64  
 12  city_missing_flag          5000 non-null   int64  
 13  state_missing_flag         5000 non-null   int64

In [29]:
df.head()

,order_id,order_date,customer_id,product_category,price,quantity,order_value,payment_method,city,state,customer_age,customer_age_missing_flag,city_missing_flag,state_missing_flag
0,1,2024-02-02,2616,Sports,269,50,13385,credit_card,New York,NY,66.0,0,0,0
1,2,2024-10-10,1736,Grocery,66,0,660,debit_card,Los Angeles,CA,24.0,0,0,0
2,3,2024-08-27,2543,Sports,267,0,5073,credit_card,Chicago,IL,23.0,0,0,0
3,4,2024-06-09,2252,Toys,114,125,14290,credit_card,New York,NY,70.0,0,0,0
4,5,2024-06-07,1583,Fashion,729,16,11754,credit_card,Houston,TX,75.0,0,0,0
